In [1]:
# IMPORTING LIBRARIES
import pandas as pd 
from bs4 import BeautifulSoup
import numpy as np
import requests
import time
import os
import re
from urllib.request import Request, urlopen
import ssl
import pickle
import wget
import openpyxl 
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image 
import PIL 

In [2]:
#GETTING THE FIRST PAGE
context = ssl._create_unverified_context()
start_url='https://censusindia.gov.in/2011census/HLO/HL_PCA/'

res = urlopen(start_url+"Houselisting-housing-HLPCA.html",context=context)
soup = BeautifulSoup(res, "html.parser")

In [120]:
#GETTING THE SUB PAGES OF ALL THE STATES AND SAVING IN A DICTIONARY
states={}
for i in soup.find_all('a'):
    states[i.get_text(strip=True)]=urlopen(start_url+str(i['href']),context=context)

In [121]:
#PARSING ABOVE DATA USING BEAUTIFUL SOUP 
state_soup={}
for i,j in states.items():
    soup = BeautifulSoup(j, "html.parser")
    state_soup[i]=soup
#STORING THE DATA AS A PICKLE FILE
with open('state_soup.pickle', 'wb') as f:
    pickle.dump(state_soup, f, protocol=pickle.HIGHEST_PROTOCOL)    

In [122]:
#DEFINING A FUNCTION 
def csv_downloader(state_name,city_name,d_link):
    cs=requests.get(d_link,verify=False)  #DOWNLOADING DATA OF EACH CITY
    city_name_xls=str(city_name)+'.xls' 
    output = open(city_name_xls, 'wb') #SAVING EACH CITY DATA ON DISK
    output.write(cs.content)
    output.close()
    
    temp_city_csv=pd.read_excel(city_name_xls,engine='openpyxl',header=[3,4,5,6]) #READING THE SAVED EXCEL DATA AS PANDAS DATAFRAME
    os.remove(city_name_xls)  #REMOVING THE FILE ON DISK
    print(state_name,'->',city_name,'->',temp_city_csv.shape)
    return temp_city_csv

In [128]:
# MAIN CODE TO GENERATE THE FINAL DATAFRAME
warnings.filterwarnings("ignore")
country_csv=pd.DataFrame()  #EMPTY DATAFRAME

for i,j in state_soup.items():
    for k in j.find_all('a'):
        if i not in ['Manipur','Madhya Pradesh']:  #LEAVING THESE CITIES
            d_link=str(start_url+str(k['href']))   #GETTING DOWNLOAD LINK
            temp_try=csv_downloader(i,k.get_text(strip=True),d_link)
            country_csv=pd.concat([country_csv,temp_try],axis=0)  #CONCATENATING THE RETURNED CITY DATA WITH PREVIOUS CITY

#SEPARATE FILE FOR 2 CITIES

two_country_csv=pd.DataFrame()
for i,j in state_soup.items():
    for k in j.find_all('a'):
        if i in ['Manipur','Madhya Pradesh']:
            d_link=str(start_url+str(k['href']))
            temp_try=csv_downloader(i,k.get_text(strip=True),d_link)
            two_country_csv=pd.concat([two_country_csv,temp_try],axis=0)

Andaman and Nicobar Islands UT -> Nicobars -> (81, 145)
Andaman and Nicobar Islands UT -> North  & Middle Andaman -> (203, 145)
Andaman and Nicobar Islands UT -> South Andaman -> (144, 145)
Haryana -> Ambala -> (583, 145)
Haryana -> Bhiwani -> (562, 145)
Haryana -> Faridabad -> (194, 145)
Haryana -> Fatehabad -> (321, 145)
Haryana -> Gurgaon -> (348, 145)
Haryana -> Hisar -> (390, 145)
Haryana -> Jhajjar -> (333, 145)
Haryana -> Jind -> (417, 145)
Haryana -> Kaithal -> (359, 145)
Haryana -> Karnal -> (565, 145)
Haryana -> Kurukshetra -> (505, 145)
Haryana -> Mahendragarh -> (446, 145)
Haryana -> Mewat -> (491, 145)
Haryana -> Palwal -> (351, 145)
Haryana -> Panchkula -> (292, 145)
Haryana -> Panipat -> (257, 145)
Haryana -> Rewari -> (472, 145)
Haryana -> Rohtak -> (225, 145)
Haryana -> Sirsa -> (442, 145)
Haryana -> Sonipat -> (427, 145)
Haryana -> Yamunanagar -> (710, 145)
Nagaland -> Dimapur -> (295, 145)
Nagaland -> Kiphire -> (124, 145)
Nagaland -> Kohima -> (158, 145)
Nagaland ->

In [148]:
#REMOVING UNNECESSARY COLUMNS FROM THE "two_country_csv" 
two_country_csv.drop(two_country_csv.iloc[:,145:].columns,axis=1,inplace=True)

#MAKING FINAL DATAFRAME AFTER CONCATENATION
final_df=pd.concat([country_csv,two_country_csv],axis=0)

#SAVING "final_df" AS CSV ON DISK
final_df.to_csv('final_df.csv')

In [ ]:
# EXPERIMENTAL : Scraping population of each state to see its effect on amneties
wiki= urlopen('https://en.wikipedia.org/wiki/2011_Census_of_India')
wiki_soup=BeautifulSoup(wiki,'html.parser')
wiki_df={}
t_list=[]
title=[]
for i,j in enumerate(wiki_soup.find_all('table',{'class':'wikitable sortable collapsible mw-datatable'})[0]('th')):
            if i<15 and i>0:
                title.append(j.get_text(strip=True))
                
for i,j in enumerate(wiki_soup.find_all('table',{'class':'wikitable sortable collapsible mw-datatable'})[0]('td')):
    if i in np.arange(0,1000,15):
        continue
    if j.get_text(strip=True)== 'Andhra Pradesh[a]':
        t_list.append('Andhra Pradesh')
        continue
    if j.get_text(strip=True)== 'Delhi':
        t_list.append('Nct Of Delhi')
        continue
    t_list.append(j.get_text(strip=True))

    if len(t_list)==14:
        wiki_df[i//15+1]=t_list
        t_list=[]
pop_df_main=pd.DataFrame(wiki_df,index=None).T.set_axis(title, axis=1)
pop_df=pop_df_main.sort_values('State /Union Territory(UT)')['Population'].apply(lambda x:int(x.replace(',',''))).values

# Incorporating population produced unfavourable effects 

In [2]:
# reading the csv file
df=pd.read_csv('final_df.csv',header=[0,1,2,3])

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Scraping of Latitude and Longitude for all districts in the census - to be used in web app

In [3]:
#Districts to be scraped
dist=(df[('District Name', 'Unnamed: 3_level_1', 'Unnamed: 3_level_2', '4')]+' '+df[('State Name', 'Unnamed: 1_level_1', 'Unnamed: 1_level_2', '2')]).values

In [5]:
# Converting spaces and '&'
dist_=[]
for i in dist:
    if ' ' in i:
        a=i.replace('  ','+')
        a=a.replace(' ','+')
        a=a.replace('&','%26')
        a=a.replace(' ','+')
        dist_.append(a)
    else:
        dist_.append(i)    

['Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAMAN+%26+NICOBAR+ISLANDS',
 'Nicobars+ANDAM

In [33]:
# Generating and saving the requests
resp={}
for i,x in enumerate(dist_):
    url=f'https://www.google.com/search?q={x}+India+latitude+and+longitude'
    resp[dist[i]]=requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0'})
    time.sleep(1)
    print(dist[i])

Nicobars ANDAMAN & NICOBAR ISLANDS
North  & Middle Andaman ANDAMAN & NICOBAR ISLANDS
South Andaman ANDAMAN & NICOBAR ISLANDS
Ambala HARYANA
Bhiwani HARYANA
Faridabad HARYANA
Fatehabad HARYANA
Gurgaon HARYANA
Hisar HARYANA
Jhajjar HARYANA
Jind HARYANA
Kaithal HARYANA
Karnal HARYANA
Kurukshetra HARYANA
Mahendragarh HARYANA
Mewat  HARYANA
Palwal  HARYANA
Panchkula HARYANA
Panipat HARYANA
Rewari HARYANA
Rohtak HARYANA
Sirsa HARYANA
Sonipat HARYANA
Yamunanagar HARYANA
Dimapur  NAGALAND
Kiphire NAGALAND
Kohima NAGALAND
Longleng NAGALAND
Mokokchung NAGALAND
Mon NAGALAND
Peren NAGALAND
Phek NAGALAND
Tuensang NAGALAND
Wokha NAGALAND
Zunheboto NAGALAND
Adilabad ANDHRA PRADESH
Anantapur ANDHRA PRADESH
Chittoor ANDHRA PRADESH
East Godavari ANDHRA PRADESH
Guntur ANDHRA PRADESH
Hyderabad ANDHRA PRADESH
Karimnagar ANDHRA PRADESH
Khammam ANDHRA PRADESH
Krishna ANDHRA PRADESH
Kurnool ANDHRA PRADESH
Mahbubnagar ANDHRA PRADESH
Medak ANDHRA PRADESH
Nalgonda ANDHRA PRADESH
Nizamabad ANDHRA PRADESH
Prakasam

In [173]:
# Getting the lat, long using regex
loc_dic={}
na_loc=[]
for i in resp.keys():
    try:
        len_=re.search('class="Z0LcW"',resp[i].text).end()
        loc_dic[i]=resp[i].text[len_+1:len_+21].replace(' ','').replace('N','').replace('°','').split(',')
        # print(i,loc_dic[i])
    except:
        na_loc.append(i)

In [175]:
# updating missing values manually
loc_dic.update({'Adilabad ANDHRA PRADESH ':['19.6641','78.5320'],
 'Karimnagar ANDHRA PRADESH':['18.4386', '79.1288'],
 'Khammam ANDHRA PRADESH':['17.2473', '80.1514'],
 'Mahbubnagar ANDHRA PRADESH':['16.7488', '78.0035'],
 'Medak ANDHRA PRADESH':['18.0529', '78.2619'],
 'Rangareddy ANDHRA PRADESH':['17.3891', '77.8367'],
 'Anjaw ARUNACHAL PRADESH':['27.79866','95.90994'],
 'Kurung Kumey ARUNACHAL PRADESH':['27.986368','93.157835'],
 'Leh(Ladakh) JAMMU & KASHMIR':['34.1526', '77.5771'],
 'Lakshadweep LAKSHADWEEP':['10.5667', '72.6417'],
 'Kanshiram Nagar UTTAR PRADESH':['27.7952', '78.7930'],
 'East NCT OF DELHI':['28.6280','77.2956'],
 'North NCT OF DELHI':['28.7886', '77.1412']})

In [176]:
# Saving as a Pandas dataframe
loc_t=pd.DataFrame(loc_dic,index=['Latitude','Longitude']).T
loc_t.to_csv('lat_long.csv')
